# Financial Statement Analysis with LLM

This notebook uses an LLM to intelligently parse Excel financial statements and extract:
- Classification: Actual vs Forecast
- Company name
- Dates of figures
- Line items in each row

In [1]:
import pandas as pd
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [2]:
# Load the Excel file
excel_file_path = 'dataset/nycfine_FS.xlsx'

# Read all sheets to get complete context
excel_file = pd.ExcelFile(excel_file_path)
sheet_names = excel_file.sheet_names
print(f"Available sheets: {sheet_names}")

# Read both actual and forecast sheets
df_actual = pd.read_excel(excel_file_path, sheet_name='actual')
df_forecast = pd.read_excel(excel_file_path, sheet_name='forecast')

print(f"\nActual sheet shape: {df_actual.shape}")
print(f"Forecast sheet shape: {df_forecast.shape}")
print(f"\nFirst few rows of actual:")
print(df_actual.head(10))
print(f"\nFirst few rows of forecast:")
print(df_forecast.head(10))

Available sheets: ['actual', 'forecast']

Actual sheet shape: (27, 8)
Forecast sheet shape: (27, 8)

First few rows of actual:
     NYC Fine Jewelry           Unnamed: 1           Unnamed: 2  \
0    Income Statement                  NaN                  NaN   
1       For the month  2025-01-01 00:00:00  2025-02-01 00:00:00   
2                 NaN                  NaN                  NaN   
3               Sales                40000                35000   
4  Cost of goods sold                15000                13000   
5        Gross profit                25000                22000   
6                 NaN                  NaN                  NaN   
7                 NaN                  NaN                  NaN   
8  Operating expenses                  NaN                  NaN   
9                 NaN                  NaN                  NaN   

            Unnamed: 3           Unnamed: 4           Unnamed: 5  \
0                  NaN                  NaN                  NaN   

In [3]:
def analyze_financial_statement_with_llm(df):
    """
    Use LLM to analyze the financial statement and extract key information
    """
    # Convert DataFrame to a readable format for the LLM
    # Take first 20 rows to avoid token limits
    sample_data = df.head(20).to_string()
    
    prompt = f"""
    Analyze this financial statement data and extract the following information in JSON format:
    
    1. classification: "actual" or "forecast" (determine if this shows actual historical data or forecasted/projected data)
    2. company_name: The name of the company (look for company name in headers or metadata)
    3. dates: List of all dates/periods shown in the data (could be months, quarters, years)
    4. line_items: List of all financial line items/categories found in rows (e.g., Revenue, Expenses, etc.)
    
    Financial Statement Data:
    {sample_data}
    
    Please respond with valid JSON only:
    {{
        "classification": "actual" or "forecast",
        "company_name": "Company Name",
        "dates": ["date1", "date2", ...],
        "line_items": ["item1", "item2", ...]
    }}
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a financial analyst expert. Analyze financial statements and extract key information accurately. Always respond with valid JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1
        )
        
        # Parse the JSON response
        result = json.loads(response.choices[0].message.content)
        return result
        
    except Exception as e:
        print(f"Error in LLM analysis: {e}")
        return None

In [4]:
# Analyze both actual and forecast sheets
actual_result = analyze_financial_statement_with_llm(df_actual)
forecast_result = analyze_financial_statement_with_llm(df_forecast)

if actual_result:
    print("\n=== ACTUAL VALUES ANALYSIS ===")
    print(f"Classification: {actual_result['classification'].upper()}")
    print(f"Company Name: {actual_result['company_name']}")
    print(f"\nDates/Periods Found:")
    for date in actual_result['dates']:
        print(f"  - {date}")
    print(f"\nLine Items Found:")
    for item in actual_result['line_items']:
        print(f"  - {item}")
else:
    print("Failed to analyze the actual values")

if forecast_result:
    print("\n=== FORECAST VALUES ANALYSIS ===")
    print(f"Classification: {forecast_result['classification'].upper()}")
    print(f"Company Name: {forecast_result['company_name']}")
    print(f"\nDates/Periods Found:")
    for date in forecast_result['dates']:
        print(f"  - {date}")
    print(f"\nLine Items Found:")
    for item in forecast_result['line_items']:
        print(f"  - {item}")
else:
    print("Failed to analyze the forecast values")


=== ACTUAL VALUES ANALYSIS ===
Classification: FORECAST
Company Name: NYC Fine Jewelry

Dates/Periods Found:
  - 2025-01-01
  - 2025-02-01
  - 2025-03-01
  - 2025-04-01
  - 2025-05-01
  - 2025-06-01
  - 2025-07-01

Line Items Found:
  - Sales
  - Cost of goods sold
  - Gross profit
  - CEO salary
  - Admin assistant salary
  - Total general and administrative expenses
  - Instagram ads
  - Marketing assistant
  - Total sales and marketing expenses

=== FORECAST VALUES ANALYSIS ===
Classification: FORECAST
Company Name: NYC Fine Jewelry

Dates/Periods Found:
  - 2025-01-01
  - 2025-02-01
  - 2025-03-01
  - 2025-04-01
  - 2025-05-01
  - 2025-06-01
  - 2025-07-01

Line Items Found:
  - Sales
  - Cost of goods sold
  - Gross profit
  - Operating expenses
  - General and administrative expenses
  - CEO salary
  - Admin assistant salary
  - Total general and administrative expenses
  - Sales and marketing expenses
  - Instagram ads
  - Marketing assistant
  - Total sales and marketing expen

In [5]:
def extract_detailed_line_items(df):
    """
    Use LLM to extract detailed information about each line item
    """
    # Convert entire DataFrame to string for comprehensive analysis
    data_str = df.to_string()
    
    prompt = f"""
    Analyze this financial statement and extract detailed information about each line item.
    For each row that contains financial data, identify:
    1. The line item name/description
    2. The values for each period/date
    3. The category it belongs to (e.g., Revenue, Operating Expenses, etc.)
    
    Financial Data:
    {data_str}
    
    Please respond with a JSON array where each object represents a line item:
    [
        {{
            "line_item": "Item name",
            "category": "Financial category",
            "values": {{"period1": value1, "period2": value2, ...}}
        }},
        ...
    ]
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a financial analyst. Extract detailed line item information from financial statements. Always respond with valid JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1
        )
        
        result = json.loads(response.choices[0].message.content)
        return result
        
    except Exception as e:
        print(f"Error in detailed analysis: {e}")
        return None

In [6]:
# Extract detailed line items from both sheets
actual_items = extract_detailed_line_items(df_actual)
forecast_items = extract_detailed_line_items(df_forecast)

if actual_items:
    print("\n=== ACTUAL VALUES ===")
    for item in actual_items:
        print(f"\nLine Item: {item['line_item']}")
        print(f"Category: {item['category']}")
        print(f"Values: {item['values']}")
        print("-" * 50)
else:
    print("Failed to extract actual values")

if forecast_items:
    print("\n=== FORECAST VALUES ===")
    for item in forecast_items:
        print(f"\nLine Item: {item['line_item']}")
        print(f"Category: {item['category']}")
        print(f"Values: {item['values']}")
        print("-" * 50)
else:
    print("Failed to extract forecast values")


=== ACTUAL VALUES ===

Line Item: Sales
Category: Revenue
Values: {'2025-01-01': 40000, '2025-02-01': 35000, '2025-03-01': 10000, '2025-04-01': 53000, '2025-05-01': 73000, '2025-06-01': 81500, '2025-07-01': 68000}
--------------------------------------------------

Line Item: Cost of goods sold
Category: Cost of Sales
Values: {'2025-01-01': 15000, '2025-02-01': 13000, '2025-03-01': 5000, '2025-04-01': 21500, '2025-05-01': 29000, '2025-06-01': 35000, '2025-07-01': 27000}
--------------------------------------------------

Line Item: Gross profit
Category: Profit
Values: {'2025-01-01': 25000, '2025-02-01': 22000, '2025-03-01': 5000, '2025-04-01': 31500, '2025-05-01': 44000, '2025-06-01': 46500, '2025-07-01': 41000}
--------------------------------------------------

Line Item: CEO salary
Category: General and administrative expenses
Values: {'2025-01-01': 5000, '2025-02-01': 7000, '2025-03-01': 5000, '2025-04-01': 6000, '2025-05-01': 5000, '2025-06-01': 5000, '2025-07-01': 5000}
-------

In [7]:
# Save actual and forecast values to separate JSON files
if actual_result and actual_items:
    actual_analysis = {
        "summary": actual_result,
        "detailed_line_items": actual_items,
        "raw_data_shape": df_actual.shape,
        "sheet_name": "actual"
    }
    
    with open('actual_values.json', 'w') as f:
        json.dump(actual_analysis, f, indent=2)
    
    print("\nActual values saved to 'actual_values.json'")
    print(f"Total actual line items extracted: {len(actual_items)}")

if forecast_result and forecast_items:
    forecast_analysis = {
        "summary": forecast_result,
        "detailed_line_items": forecast_items,
        "raw_data_shape": df_forecast.shape,
        "sheet_name": "forecast"
    }
    
    with open('forecast_values.json', 'w') as f:
        json.dump(forecast_analysis, f, indent=2)
    
    print("\nForecast values saved to 'forecast_values.json'")
    print(f"Total forecast line items extracted: {len(forecast_items)}")


Actual values saved to 'actual_values.json'
Total actual line items extracted: 13

Forecast values saved to 'forecast_values.json'
Total forecast line items extracted: 11


In [8]:
# Display the raw data for reference
print("\n=== RAW DATA PREVIEW ===")
print(f"Actual data shape: {df_actual.shape}")
print(f"Forecast data shape: {df_forecast.shape}")
print("\nActual dataset:")
print(df_actual)
print("\nForecast dataset:")
print(df_forecast)


=== RAW DATA PREVIEW ===
Actual data shape: (27, 8)
Forecast data shape: (27, 8)

Actual dataset:
                             NYC Fine Jewelry           Unnamed: 1  \
0                            Income Statement                  NaN   
1                               For the month  2025-01-01 00:00:00   
2                                         NaN                  NaN   
3                                       Sales                40000   
4                          Cost of goods sold                15000   
5                                Gross profit                25000   
6                                         NaN                  NaN   
7                                         NaN                  NaN   
8                          Operating expenses                  NaN   
9                                         NaN                  NaN   
10        General and administrative expenses                  NaN   
11                                 CEO salary                

In [ ]:
# Calculate variances by loading and subtracting JSON values
def calculate_variances_from_json():
    """
    Calculate variances by deducting forecast JSON values from actual JSON values
    For Revenue/Income: Variance = Actual - Forecast (positive = favorable)
    For Expenses: Variance = Forecast - Actual (positive = favorable, actual < forecast)
    """
    try:
        # Load actual values JSON
        with open('actual_values.json', 'r') as f:
            actual_data = json.load(f)
        
        # Load forecast values JSON  
        with open('forecast_values.json', 'r') as f:
            forecast_data = json.load(f)
        
        actual_items = actual_data['detailed_line_items']
        forecast_items = forecast_data['detailed_line_items']
        
        variances = []
        
        # Create lookup dictionary for forecast items
        forecast_lookup = {item['line_item']: item for item in forecast_items}
        
        # Define expense categories
        expense_categories = [
            'Cost of Sales', 'Operating expenses', 'General and administrative expenses', 
            'Sales and marketing expenses', 'Tax'
        ]
        
        for actual_item in actual_items:
            line_item = actual_item['line_item']
            category = actual_item['category']
            
            # Find matching forecast item
            if line_item in forecast_lookup:
                forecast_item = forecast_lookup[line_item]
                
                variance_data = {
                    "line_item": line_item,
                    "category": category,
                    "actual_values": actual_item['values'],
                    "forecast_values": forecast_item['values'],
                    "variances": {}
                }
                
                # Calculate variance for each period
                for period in actual_item['values']:
                    if period in forecast_item['values']:
                        try:
                            actual_val = float(actual_item['values'][period]) if actual_item['values'][period] not in [None, '', 'NaN'] else 0
                            forecast_val = float(forecast_item['values'][period]) if forecast_item['values'][period] not in [None, '', 'NaN'] else 0
                            
                            # For expenses: Variance = Forecast - Actual (positive means favorable)
                            # For revenue/income: Variance = Actual - Forecast (positive means favorable)
                            if category in expense_categories:
                                variance = forecast_val - actual_val
                            else:
                                variance = actual_val - forecast_val
                                
                            variance_data["variances"][period] = variance
                        except (ValueError, TypeError):
                            print(f"Could not compute variance for {line_item} in {period}")
                            variance_data["variances"][period] = None
                
                variances.append(variance_data)
            else:
                print(f"No matching forecast found for actual line item: {line_item}")
        
        return variances
        
    except FileNotFoundError as e:
        print(f"Error: Could not find JSON file - {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format - {e}")
        return None

# Calculate variances from JSON files
print("\n=== CALCULATING VARIANCES FROM JSON FILES ===")
variances = calculate_variances_from_json()

if variances:
    # Save variances to JSON file
    with open('variances.json', 'w') as f:
        json.dump(variances, f, indent=2)
    
    print(f"Variances calculated for {len(variances)} line items")
    print("Variances saved to 'variances.json'")
    
    # Display variance summary
    print("\n=== VARIANCE SUMMARY ===")
    for item in variances:
        if item['variances']:
            print(f"\nLine Item: {item['line_item']}")
            print(f"Category: {item['category']}")
            for period, variance in item['variances'].items():
                if variance is not None:
                    print(f"  {period}: {variance:,.2f}")
                else:
                    print(f"  {period}: Unable to compute")
else:
    print("Failed to calculate variances from JSON files")

In [ ]:
# Calculate variance percentages by dividing variances JSON by actual values JSON
def calculate_variance_percentages():
    """
    Calculate variance percentages by dividing variances JSON by actual values JSON
    Variance Percentage = (Variances JSON / Actual Values JSON) * 100
    """
    try:
        # Load variances JSON
        with open('variances.json', 'r') as f:
            variances_data = json.load(f)
        
        # Load actual values JSON
        with open('actual_values.json', 'r') as f:
            actual_data = json.load(f)
        
        actual_items = actual_data['detailed_line_items']
        
        # Create lookup dictionary for actual items
        actual_lookup = {item['line_item']: item for item in actual_items}
        
        variance_percentages = []
        
        for variance_item in variances_data:
            line_item = variance_item['line_item']
            
            # Find matching actual item
            if line_item in actual_lookup:
                actual_item = actual_lookup[line_item]
                
                percentage_data = {
                    "line_item": line_item,
                    "category": variance_item['category'],
                    "actual_values": actual_item['values'],
                    "variances": variance_item['variances'],
                    "variance_percentages": {}
                }
                
                # Calculate variance percentage for each period
                for period in variance_item['variances']:
                    if period in actual_item['values']:
                        try:
                            variance = float(variance_item['variances'][period]) if variance_item['variances'][period] not in [None, '', 'NaN'] else 0
                            actual_val = float(actual_item['values'][period]) if actual_item['values'][period] not in [None, '', 'NaN'] else 0
                            
                            if actual_val != 0:
                                percentage = (variance / actual_val) * 100
                                percentage_data["variance_percentages"][period] = percentage
                            else:
                                percentage_data["variance_percentages"][period] = None
                                print(f"Cannot calculate percentage for {line_item} in {period}: actual value is zero")
                        except (ValueError, TypeError):
                            print(f"Could not compute variance percentage for {line_item} in {period}")
                            percentage_data["variance_percentages"][period] = None
                
                variance_percentages.append(percentage_data)
            else:
                print(f"No matching actual values found for variance line item: {line_item}")
        
        return variance_percentages
        
    except FileNotFoundError as e:
        print(f"Error: Could not find JSON file - {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format - {e}")
        return None

# Calculate variance percentages from separate JSON files
print("\n=== CALCULATING VARIANCE PERCENTAGES FROM JSON FILES ===")
variance_percentages = calculate_variance_percentages()

if variance_percentages:
    # Save variance percentages to JSON file
    with open('variance_percentages.json', 'w') as f:
        json.dump(variance_percentages, f, indent=2)
    
    print(f"Variance percentages calculated for {len(variance_percentages)} line items")
    print("Variance percentages saved to 'variance_percentages.json'")
    
    # Display variance percentage summary
    print("\n=== VARIANCE PERCENTAGE SUMMARY ===")
    for item in variance_percentages:
        if item['variance_percentages']:
            print(f"\nLine Item: {item['line_item']}")
            print(f"Category: {item['category']}")
            for period, percentage in item['variance_percentages'].items():
                if percentage is not None:
                    print(f"  {period}: {percentage:.2f}%")
                else:
                    print(f"  {period}: Unable to compute")
else:
    print("Failed to calculate variance percentages from JSON files")